# Implement alternate GMICE for `Stations_{Source}.csv`

## Configuration

In [ ]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt

from gmice import gmice

## User input
Specify Stations source and desired GMICE

Available GMICE are listed below, based on its generic functional form (i.e., not including additional metrics such as Mw, R)

Simplified predictive equations (MMI = a + b log Y):
* `PanjamaniEtAl2016`: Himalayan region; supports PGA, PGV, SA(0.3), SA(1.0), SA(2.0), SA(3.0)
* `TianEtAl2021`: GMICE for China; supports PGA, PGV, SA(0.3), SA(1.0), SA(2.0), SA(3.0)
* `AhmadzadehEtAl2020`: Iran; supports PGA, PGV
* `CataldiEtAl2021`: Italy; supports PGA, PGV, SA(0.3), SA(1.0), SA(3.0)

Piecewise equations (not yet implemented):
* `CaprioEtAl2015`: Global; supports PGA, PGV
* `MoratalaEtAl2021`: New Zealand; supports PGA, PGV
* `MontalvoArrietaEtAl2019`: Mexico (Puebla–Morelos earthquake); supports PGA, SA(1.0), SA(2.0), SA(3.0)

The GMICE should be selected based on the similarity to your region in terms of prevalent construction practices and tectonic region.

<!-- Other functional forms (not yet implemented):
* `GomezCaperaEtAl2020`: Italy;

Coefficients unavailable:
* `CiliaBaker2018`: Chilean Megathrust Earthquakes

Dependent on R and Mw (not yet implemented):
* There are many from the same references as above, but they requires additional inputs and the desired R metric may not be readily available in the station data -->




In [ ]:
# Event name
event = '20030521_M6.8_Boumerdes'
# Station source
source = "USGS"
# GMICE name
gmice_name = "CataldiEtAl2021"

# Arrange station path
station_paths = glob.glob(
    os.path.join('..', '*', event, 'Recording_Stations', 
                 f'Stations_{source}.csv'))
station_path = [file_path for file_path in station_paths if file_path.endswith(f"Stations_{source}.csv")][0]
print(f"Will modify {station_path}")

# Read station path
stations = pd.read_csv(station_path, encoding="utf-8")

# Preview
stations.head(5)

In [ ]:
# Apply selected GMICE
out, imts = gmice(gmice_name, stations)

# Determine idx, jdx
idx = stations.STATION_TYPE == "macroseismic"
jdx = out.STATION_TYPE == "macroseismic"

# Determine adjusted items
print(f"The following imts were adjusted for the {idx.sum():,.0f} macroseismic stations: {imts}")

# Plot results
for imt in imts:
    fig, ax = plt.subplots()
    if f'{imt}_VALUE' in stations.columns:
        stations[idx].plot.scatter(x="MMI_VALUE", y=f'{imt}_VALUE', ax=ax,
                            color='k', label="Original values")
    out[jdx].plot.scatter(x="MMI_VALUE", y=f'{imt}_VALUE', ax=ax,
                          color='b', label="Adjusted values")
    ax.set(title=f"GMICE: {gmice_name}")
    plt.show()

# Preview results
out[idx].head(5)

In [ ]:
# Write to new file
out_file = f"Stations_{source}_GMICE.csv"
out_path = os.path.join(os.path.dirname(station_path), out_file)
out.to_csv(out_path, index=False)
print(f"Wrote {out_path}\n",
      f"{idx.sum():,.0f} macroseismic stations with modified IMT values: {imts}")